# Section 1: Loading the train and test Data-sets

In [ ]:
import numpy as np
import pandas as pd
from IPython.display import Markdown, display
from sklearn import impute
from sklearn.preprocessing import OneHotEncoder
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import f1_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import RandomizedSearchCV
import numpy as np
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
import numpy as np

In [ ]:
#loading the train and test datasets  from UCI website
Original_train_dataset = pd.read_csv('adult.txt', delimiter=',\s',na_values="?",engine='python',header= None)
Original_test_dataset = pd.read_csv('adult_test.txt', delimiter= ',\s',na_values="?",engine='python',header=1)

#printing only 5 rows from train and test dataset
print(Original_train_dataset.head(5))
print(Original_test_dataset.head(5))


In [ ]:
#since there are no column names, adding them manually for convenient understanding.

Original_train_dataset.columns = ["age","workclass","fnlwgt","education","education-num","marital-status","occupation",
"relationship","race","sex","capital-gain","capital-loss","hours-per-week","native-country","income"]

# it was seen that the test data had a "full stop" at end of each line. hence removing it here 
Original_test_dataset.columns = ["age","workclass","fnlwgt","education","education-num","marital-status","occupation",
"relationship","race","sex","capital-gain","capital-loss","hours-per-week","native-country","income"]
Original_test_dataset['income'].replace(regex=True,inplace=True,to_replace=r'\.',value=r'')

#printing only one row to see if column names are proper:

print("train dataset is:",Original_train_dataset.head(1))
print("test dataset is :",Original_test_dataset.head(1))

In [ ]:
#Here am joining both the train and test datafranes together. this is to complete pre-processing of data toghether.
adult_df = pd.concat([Original_train_dataset,Original_test_dataset],sort=False)
print(adult_df.head(5))
print(adult_df.tail(5))

In [ ]:
#checking the shape of adult_df which has both the train and test data.
print("shape of the combined dataset is:","\n",adult_df.shape)

# Section 2:  Visualization and Pre-processing of data.
## Section 2.1 : Visualization of Train and test Features.
### Section 2.1.1 :  Checking the infomation of dataframe

In [ ]:
#checking the information of this dataframe to see if pre-processing is necessary 
print(adult_df.info())

### Section 2.1.2:Checking the summary statistics for both Numerical and Categorical features seperately:

In [ ]:
# checking the summary statistics of the adult_df 
adult_df.describe() # this will give the statistics of only the Numerical Features.

From the above statistics for numerical features it is clearly seen that, there are no missing values in any of the features. All the features have total of 48841 rows with values.

In [ ]:
adult_df.describe(include = ['O'])# Gives statistics of categorical features

The statistics of Categorical features shows that, there are few missing values in feature workclass, occupation and Native country. Next step is to find out how many values are missing in each of the features. But before finding the number of missing values in each feature, performing a simple comparison of each categorical feature with Income class to see if we can drop any of the feature.

### Section 2.1.3: Comparing the "Income" label class with other categorical features.

In [ ]:
#using seaborn, am trying to count the occurances of each values.
print("Graph1:",sns.countplot(y='occupation', hue='income', data=adult_df))

In [ ]:
print("Graph 2:",sns.countplot(y='education', hue='income', data=adult_df))

In [ ]:
print("Graph 3:",sns.countplot(y='sex', hue='income', data=adult_df))

In [ ]:
print("Graph 4:",sns.countplot(y='race', hue='income', data=adult_df))

In [ ]:
print("Graph 5:",sns.countplot(y='workclass', hue='income', data=adult_df))

In [ ]:
print("Graph 6:",sns.countplot(y='native-country', hue='income', data=adult_df))

In [ ]:
print("Graph 7:",sns.countplot(y='relationship', hue='income', data=adult_df))

In [ ]:
print("Graph 8:",sns.countplot(y='marital-status', hue='income', data=adult_df))

The inference from the all the above 8 graphs are as followed:                                                                 1. Around 90% of values in native country belongs to United States
    2. Around 85% of values in race feature are White.
    3. Almost all the categorical features have relationship with the label class "INCOME". Hence we cannot decide to drop any of the features now. This leads us to carry out the process of feature  

### Section 2.1.4: Checking for missing value count in each feature

In [ ]:
print("total number of nan present before imputations" '\n',adult_df.isna().sum().sort_values(ascending=False).head())

## Section 2.2: Preprocessing of data:

In this section, the complete process of preprocessing and preparing the data for feeding to classifer models will be carried out extensively.

### Section 2.2.1: Imputation using Simple imputer
Here am using simple imputer funtion to impute the missing values in the 3 categorical feature obtained in section 2.1.4. The missing values will be replaced by most frequently available values in each column.

In [ ]:
imputeclass = impute.SimpleImputer(missing_values=np.nan,strategy='most_frequent')
adult_df[["workclass"]] = imputeclass.fit_transform(adult_df[["workclass"]])
adult_df[["occupation"]] = imputeclass.fit_transform(adult_df[["occupation"]])
adult_df[["native-country"]] = imputeclass.fit_transform(adult_df[["native-country"]])
print("total number of nan present after imputations" '\n',adult_df.isna().sum().sort_values(ascending=False).head())

## Section 2.2.2 : Handling the label class ["Income"] seperately. 

In [ ]:
# here am replacing >50K with 1 and <50K with 0
print(adult_df['income'].value_counts())
# print(adult_df['income'])

In [ ]:
adult_df['income'] = [0 if x == '<=50K' else 1 for x in adult_df['income']]
print(adult_df['income'].value_counts())

### Section 2.2.3: Using one-hot encoder, changing all the categorical features to numerical features.


In [ ]:
#applying one-hot encoder to each categorical feature one by one

Encoder = OneHotEncoder(sparse=False,categories='auto')
adult_df['workclass'] = Encoder.fit_transform(adult_df[['workclass']])
# train_feature[['education','marital-status']] = Encoder.fit_transform(train_feature[['education','marital-status']])
adult_df['education'] = Encoder.fit_transform(adult_df[['education']])
adult_df['marital-status'] = Encoder.fit_transform(adult_df[['marital-status']])
adult_df['occupation'] = Encoder.fit_transform(adult_df[['occupation']])
adult_df['race'] = Encoder.fit_transform(adult_df[['race']])
adult_df['relationship'] = Encoder.fit_transform(adult_df[['relationship']])
adult_df['sex'] = Encoder.fit_transform(adult_df[['sex']])


# adult_df['native-country'] = Encoder.fit_transform(adult_df[['native-country']])


In [ ]:
adult_df['sex']

In [ ]:
#checking the information of the adult_df after pre-processing
print(adult_df.info())

### checking the native-country feature and converting them seperately.
It can be clearly seen now from the information that all the categorical features are transformed to numerical feature except the feature "Native-country" 

In [ ]:
#checking for unique values in column native-country
uniq = adult_df['native-country'].unique()
print(uniq)
print(len(uniq))

In [ ]:
#converting all the countries which are not UnitedStates to "OTHERS"
adult_df['native-country']= ['United-States' if x == 'United-States' else 'Others' for x in adult_df['native-country']] 
uniq = adult_df['native-country'].unique()
print(uniq)
print(len(uniq))

In [ ]:
#here the categorical native-country feature is converted to numerical feature.
adult_df['native-country'] = Encoder.fit_transform(adult_df[['native-country']])

In [ ]:
print(adult_df.info())

Therefore, i have converted all the categorical feature data to numerical feature data. Hence the data is now ready for feeding into the classifer models to be built in next section.

## Section 2.3: Splitting data and building classifier Models.

In this section, the preprocessed data will be splitted to Train feature, Test features and  Train label, Test label. 
These datasets will be then used to build various classifier models such as SVM, Naive Bayes, Decision Tree, 

### Section 2.3.1: Splitting of data into train and test 

In [ ]:
#splitting the adult dataframe into feature and 
adultdf_feature =adult_df.drop('income',axis = 1 )
adultdf_label = adult_df['income']
print(adultdf_feature.head(5))
print(adultdf_label.head(5))

In [ ]:
#using the train_test_split function, dividing the data into trainfeature, trainlabel and testfeature, testlabel
train_feature,test_feature,train_label,test_label = train_test_split(adultdf_feature, adultdf_label, test_size=0.3, random_state=0)

print("The shape of train feature is :", train_feature.shape)
print("The shape of train label is :",train_label.shape)
print("The shape of test feature is :",test_feature.shape)
print("The shape of test label is :",test_label.shape)

## 2.3.2 Building Classifier Models:


#### 1. K-nearest neighbour model

In [ ]:
#KNN model with all the features
knn = KNeighborsClassifier(n_neighbors=28)
knn.fit(train_feature, train_label)
knn_pred = knn.predict(test_feature)
# print(knn_pred)
knnaccuracy = metrics.accuracy_score(test_label,knn_pred)
print ("KNN Accuracy ", knnaccuracy)
cf_mat = confusion_matrix(test_label,knn_pred)
print("confusion matrix is :",cf_mat)
print("classification report is  : " '\n',classification_report(test_label, knn_pred))


#### 2. Gaussian NAive Bayes Model


In [ ]:
#Naive Bayes model with all the features:
nBayes = GaussianNB()
nBayes = nBayes.fit(train_feature, train_label)
Bayes_pred = nBayes.predict(test_feature)
Bayesaccuracy = metrics.accuracy_score(test_label,Bayes_pred)
print ("Naive Bayes Accuracy ", Bayesaccuracy)
cf_mat = confusion_matrix(test_label,Bayes_pred)
print("confusion matrix is :",cf_mat)
print("F1_Score : ",f1_score(test_label, Bayes_pred))
print("classification report is  : " '\n',classification_report(test_label, Bayes_pred))




#### 3. SVM Model


In [ ]:
svc = SVC()
svc = svc.fit(train_feature, train_label)
svc_pred = svc.predict(test_feature)
svcaccuracy = metrics.accuracy_score(test_label,svc_pred)
print ("SVM Accuracy ", svcaccuracy)
cf_mat = confusion_matrix(test_label,svc_pred)
print("confusion matrix is :",cf_mat)
print("F1_Score : ",f1_score(test_label, svc_pred))
print("classification report is  : " '\n',classification_report(test_label, svc_pred))

# print(svc_pred)
# print("F1_Score : ",f1_score(test_label, svc_pred))
# conf_matrix_svc = confusion_matrix(test_label,svc_pred)
# print(conf_matrix_svc)
# print(classification_report(test_label,svc_pred))

#### 4. Decision tree Model

In [ ]:
#decision Model with all the features 
decTree = DecisionTreeClassifier()
decTree = decTree.fit(train_feature, train_label)
decTree_pred = decTree.predict(test_feature)
decTreeaccuracy = metrics.accuracy_score(test_label,decTree_pred)
print ("Decision Tree Accuracy ", decTreeaccuracy)
cf_mat = confusion_matrix(test_label,decTree_pred)
print("confusion matrix is :",cf_mat)
print("F1_Score : ",f1_score(test_label, decTree_pred))
print("classification report is  : " '\n',classification_report(test_label, decTree_pred))



#### 5. Random forest classifer 

In [ ]:
#Random forest classifer with all the features
RFC=RandomForestClassifier()
RFC = RFC.fit(train_feature, train_label)
RFC_pred = RFC.predict(test_feature)
print(RFC_pred)
RFCaccuracy = metrics.accuracy_score(test_label,RFC_pred)
print ("Random Forest Accuracy ", RFCaccuracy)
conf_matrix_RFC = confusion_matrix(test_label,RFC_pred)
print(conf_matrix_RFC)
print("F1_Score : ",f1_score(test_label, RFC_pred))
print("classification report is  : " '\n',classification_report(test_label, RFC_pred))




##### It is clearly seen from the above section that all the classifer models have accuracy between the range of 75% to 85% . This is good but only accuracy cannot be considered for choosing the best classifier model. Therefore it is also infered that the F1_score is very poor for almost all the models. Hence  we will employ feature selection techniques to choose the appropriate features.

# Section 3:  Different feature selection techniques and Feature encoding techniques

### Section 3.1 Feature selection techniques

#### Section 3.1.1: Univariate Selection: 

In [ ]:
best_feature  = SelectKBest(score_func=chi2, k=10)
fit = best_feature.fit(adultdf_feature, adultdf_label)
dfscores = pd.DataFrame(fit.scores_)
dfcolumns = pd.DataFrame(adultdf_feature.columns)
featureScores = pd.concat([dfcolumns,dfscores],axis=1)
featureScores.columns = ['Specs','Score']  #naming the dataframe columns
print(featureScores.nlargest(14,'Score'))  #print 10 best features


# # np.set_printoptions(precision=3)
# # print((fit.scores_)/10000000000)
# # features = fit.transform(adultdf_feature)
# # print(features)

#### 3.1.2 : Tree Based feature selection 

In [ ]:
forest = RandomForestClassifier(n_estimators=250, random_state=0)
forest.fit(adultdf_feature, adultdf_label)
# importances contains the feature importance value for each metric
importances_train = forest.feature_importances_
print(importances_train)
# argsort returns the indices that will sort the original array
sortedIndices = np.argsort(importances_train)
print("The Indices are "'\n',sortedIndices)

In [ ]:
model = LogisticRegression()
rfe = RFE(model, 10)
fit = rfe.fit(adultdf_feature, adultdf_label)
print("Num Features: %s" % (fit.n_features_))
print("Selected Features: %s" % (fit.support_))
print("Feature Ranking: %s" % (fit.ranking_))

Therfore it is clearly seen from the above three feature selection technique that  features ‘education’, ‘fnlwgt’  are common and hence can be removed from dataset

###  Section 3.2 : Create new dataset by dropping the weakest feature

In [ ]:
# dropping the weakest features from the train_feature, test feature datasets.

train_feature_new = train_feature.drop(['education','fnlwgt'],axis = 1)
test_feature_new = test_feature.drop(['education','fnlwgt'],axis = 1)
print("The shape of train feature is :", train_feature_new.shape)
print("The shape of train label is :",train_label.shape)
print("The shape of test feature is :",test_feature_new.shape)
print("The shape of test label is :",test_label.shape)

## Section  3.3 : Create classifer models for new dataset, that is weakest features are removed.

The newly created datasets will be passed to the classifiers created in section 2.3.2 as it will be easy to compare the results.

In [ ]:
# Knn Model with weakest features dropped from the dataset
print("==================================================================")
knn_new = KNeighborsClassifier(n_neighbors=28)
knn_new.fit(train_feature_new, train_label)
knn_pred_new = knn_new.predict(test_feature_new)
# print(knn_pred)
knnaccuracy_new = metrics.accuracy_score(test_label,knn_pred_new)
print ("New KNN Accuracy ", knnaccuracy_new)
cf_mat_new = confusion_matrix(test_label,knn_pred_new)
print("New confusion matrix is :",cf_mat_new)
print("New classification report is  : " '\n',classification_report(test_label, knn_pred_new))


#==================================================

#Naive Bayes with weakest features dropped 

nBayes_new = GaussianNB()
nBayes_new = nBayes_new.fit(train_feature_new, train_label)
Bayes_pred_new = nBayes_new.predict(test_feature_new)
Bayesaccuracy_new = metrics.accuracy_score(test_label,Bayes_pred_new)
print ("Naive Bayes Accuracy ", Bayesaccuracy_new)
cf_mat_new = confusion_matrix(test_label,Bayes_pred_new)
print("confusion matrix is :",cf_mat_new)
print("F1_Score : ",f1_score(test_label, Bayes_pred_new))
print("classification report is  : " '\n',classification_report(test_label, Bayes_pred_new))

#==========================================================

#decision model with weakest features dropped from dataset 
decTree_new = DecisionTreeClassifier()
decTree_new = decTree_new.fit(train_feature_new, train_label)
decTree_pred_new = decTree_new.predict(test_feature_new)
decTreeaccuracy_new = metrics.accuracy_score(test_label,decTree_pred_new)
print ("New Decision Tree Accuracy ", decTreeaccuracy_new)
cf_mat_new = confusion_matrix(test_label,decTree_pred_new)
print("New confusion matrix is :",cf_mat_new)
print("New F1_Score : ",f1_score(test_label, decTree_pred_new))
print("New classification report is  : " '\n',classification_report(test_label, decTree_pred_new))

#==========================================================================================

#Random forest classifer with weakest feature dropped.

RFC_new=RandomForestClassifier()
RFC_new = RFC_new.fit(train_feature_new, train_label)
RFC_pred_new = RFC_new.predict(test_feature_new)
print(RFC_pred_new)
RFCaccuracy_new = metrics.accuracy_score(test_label,RFC_pred_new)
print ("New Random Forest Accuracy ", RFCaccuracy_new)
conf_matrix_RFC_new = confusion_matrix(test_label,RFC_pred_new)
print(conf_matrix_RFC_new)
print("New F1_Score : ",f1_score(test_label, RFC_pred_new))
print("New classification report is  : " '\n',classification_report(test_label, RFC_pred_new))

svc_new = SVC()
svc_new = svc_new.fit(train_feature_new, train_label)
svc_pred_new = svc_new.predict(test_feature_new)
svcaccuracy_new = metrics.accuracy_score(test_label,svc_pred_new)
print ("New SVM Accuracy ", svcaccuracy_new)
cf_mat = confusion_matrix(test_label,svc_pred_new)
print("New confusion matrix is :",cf_mat_new)
print("New F1_Score : ",f1_score(test_label, svc_pred_new))
print("New classification report is  : " '\n',classification_report(test_label, svc_pred_new))



### 3.4 Hyper-parameter Optimization:¶

#### Cross fold Validation using cross_val_score

In [ ]:
#drop the weakest columns from original dataframe so that it can be used in hyper parameter optimization.

#splitting the adult dataframe into feature and 
adultdf_featurenew =adult_df.drop(['income','education','fnlwgt'],axis = 1 )
adultdf_labelnew = adult_df['income']
print(adultdf_featurenew.shape)
print(adultdf_labelnew.shape)

In [ ]:
clf = RandomForestClassifier().fit(train_feature_new, train_label) #Initialize with whatever parameters you want to

# 10-Fold Cross validation
print (np.mean(cross_val_score(clf, adultdf_feature, adultdf_label, cv=10)))
print ("F1_score is :",np.mean(cross_val_score(clf, adultdf_feature, adultdf_label, cv=10,scoring='f1_macro')))
# clf.score(test_feature,test_label)

######  Using GridSearchCV

In [ ]:
clf = RandomForestClassifier()
param_grid = { 
    'n_estimators': [200],
    'max_features': ['auto'], #selects all the features 
    'max_depth' : [10],
    'criterion' :['entropy']
}
CV_rfc = GridSearchCV(estimator=clf, param_grid=param_grid, cv= 5)
CV_rfc.fit(train_feature_new, train_label)
CVRFC_pred = CV_rfc.predict(test_feature_new)
print(CVRFC_pred)
CVRFCaccuracy = metrics.accuracy_score(test_label,CVRFC_pred)
print(CVRFCaccuracy)
print("F1_Score : ",f1_score(test_label, CVRFC_pred))

